In [ ]:
from resorting import Sorting
import random
import numpy as np

total_side_length = 40
filled_length = 20
state_array = np.ndarray((total_side_length,total_side_length),dtype=bool)

for i in range(total_side_length):
    for j in range(total_side_length):
        state_array[i,j] = random.random() > 0.5
filled_shape = np.array([filled_length,filled_length],np.int32)

row_start = (total_side_length - filled_length) // 2
col_start = (total_side_length - filled_length) // 2
comp_zone = (row_start, row_start + filled_length, col_start, col_start + filled_length)

state_array_copy = state_array.copy()
state_array_copy2 = state_array.copy()

sorter = Sorting.Sorting()
# Sequential sorting
sequential_moves = sorter.sortSequentially(state_array_copy, (row_start, row_start + filled_length), (col_start, col_start + filled_length))
for move in sequential_moves:
    print("Move contains " + str(len(move.sites_list) - 1) + " steps with a total length of " + str(move.distance))

# Parallel sorting
parallel_moves = sorter.sortParallel(state_array_copy2, (row_start, row_start + filled_length), (col_start, col_start + filled_length))
for move in parallel_moves:
    print("Move shifts " + str(len(move.steps[0].rowSelection)) + " rows and " + \
        str(len(move.steps[0].colSelection)) + " columns for " + str(len(move.steps) - 1) + " steps")

Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 2.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 8.0
Move contains 1 steps with a total length of 8.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 5.0
Move contains 1 steps with a total length of 5.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 4.0
Move contains 1 steps with a total length of 5.0
Move contains 1 steps with a total length of 6.0
Move contains 1 steps with a total length of 7.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 step